<a href="https://colab.research.google.com/github/leejooan/Image-Recognition-and-Ocr/blob/master/Image_Recognition_and_Ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

바이오메디컬인공지능 프로젝트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow
!pip install webcolors
!pip install pytesseract
!pip install opencv-python
!sudo apt install tesseract-ocr
!sudo apt install tesseract-ocr-kor
!pip install webcolors

In [ ]:
import re
import pytesseract
from PIL import Image
from google.colab.patches import cv2_imshow
import webcolors
from tensorflow.python.keras.models import load_model
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
import cv2
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import os
from keras.applications.vgg16 import VGG16
vgg16 = keras.applications.vgg16
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings('ignore')
import matplotlib.image as img
from scipy.cluster.vq import whiten
%matplotlib inline
from scipy.cluster.vq import kmeans,vq

Data load

In [ ]:
image_path="/content/drive/MyDrive/img_00000004.jpg"

옷 카테고리 분류 및 색 분류, 가격 출력

In [ ]:
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
filename = "{}.jpg".format(os.getpid())  
cv2.imwrite(filename, gray)
text = pytesseract.image_to_string(Image.open(filename), lang = 'eng+kor' )
os.remove(filename)

price = text.split('\n')

price = text.split('\n')

price_list = []
for word in price:
    if word.startswith("W") or  word.startswith("₩") or word.startswith("PRICE") or  word.startswith("price") or  word.startswith("가격") or  word.startswith("판매가") or word.startswith("가"):
        price_list.append(word)

Price = ""

for P in price_list:
  S = re.findall("\d+",P) 
  Price = "".join(S) 

path="/content/drive/MyDrive/mnist_mlp_model.h5"
model = load_model(path)
from keras.preprocessing import image
image = image.load_img(image_path)
im = image.resize((224, 224))

prediction_scores = model.predict(np.expand_dims(im, axis=0))
predicted_index = np.argmax(prediction_scores)

if predicted_index ==0:
       label="Blazer"
elif predicted_index ==1:
       label="Blouse"
elif predicted_index ==2:
     label="Cardigan"
elif predicted_index ==3:
     label="Dress"
elif predicted_index ==4:
     label="Jacket"
elif predicted_index ==5:
     label="Jeans"
elif predicted_index ==6:
    label="Shorts"
elif predicted_index ==7:
    label="Skirt"
elif predicted_index ==8:
    label="Sweater"
elif predicted_index ==9:
    label="Tee"
else:
    print("Predicted label: 옷이 아닙니다.")

img2 = cv2.imread(image_path)
mask = np.zeros(img2.shape[:2],np.uint8)

image = img.imread(image_path)

bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)

rect = (50,50,450,290)
cv2.grabCut(img2,mask,rect,bgdModel,fgdModel,1,cv2.GC_INIT_WITH_RECT)

newmask = cv2.imread('./data/newmask2.png',0)
mask[newmask == 0] = 0
mask[newmask == 255] = 1
cv2.grabCut(img2,mask,None,bgdModel,fgdModel,1,cv2.GC_INIT_WITH_MASK)
mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
img2 = img2*mask2[:,:,np.newaxis]

# 이미지에서 평균 rgb값 구하기
Red = []
Green = []
Blue = []

for x in image:
    for y in x:
        Red.append(y[0])
        Green.append(y[1])
        Blue.append(y[2])

R_avg = sum(Red) / len(Red)
G_avg = sum(Green) / len(Green)
B_avg = sum(Blue) / len(Blue)

# print("Avg Value:",R_avg,G_avg,B_avg)

# Euclidean distance로 가장 근접한 색 찾기
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

requested_colour = (R_avg, G_avg, B_avg)
actual_name, closest_name = get_colour_name(requested_colour)

plt.imshow(im)
text = text.strip()
if str.isdigit(Price) == True:
   print("이 상품의 가격은",Price + "원입니다.")
    
else:
    print(closest_name, label)


In [ ]:
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
filename = "{}.jpg".format(os.getpid())  
cv2.imwrite(filename, gray)
text = pytesseract.image_to_string(Image.open(filename), lang = 'eng+kor' )
os.remove(filename)

price = text.split('\n')

price_list = []
for word in price:
    if word.startswith("W") or  word.startswith("₩") or word.startswith("PRICE") or  word.startswith("price") or  word.startswith("가격") or  word.startswith("판매가") or word.startswith("가"):
        price_list.append(word)

Price = ""

for P in price_list:
  S = re.findall("\d+",P) 
  Price = "".join(S) 
  print("이 상품의 가격은",Price + "원입니다.")

In [ ]:
if Price == None:
    print("Predicted label: ",predicted_index)

In [ ]:
print("Predicted label: ",predicted_index)

In [ ]:
# reference
# (OCR가격표인식)https://bhadreshpsavani.medium.com/how-to-use-tesseract-library-for-ocr-in-google-colab-notebook-5da5470e4fe0